# `11-validate-models`

In [1]:
import pandas, numpy, pathlib

from skops.io import load

from sklearn.utils import resample

import matplotlib.pyplot as plt

from misc import construct_line

from statsmodels.stats.weightstats import ztest as ztest

pathlib.Path('pdf/figure-5').mkdir(exist_ok=True)

number_of_bootstraps = 10

### Model validation on `validation` and `mic` sets

As discussed in the manuscript we have two further datasets (`validation` & `mic`) to evaluate the models on.

First let's load the trained models, the datasets and the results for Suspect-PZA


In [2]:
best_model = {}

for model in ['LR', 'NN', 'XB']:
    best_model[model] = load('models/'+model.lower()+'.skops', trusted=True)

X={}
Y={}
Z={}

for i in ['validation-samples', 'validation-mutations', 'mic']:
    X[i]={}
    Y[i]={}
    Z[i]={}
    with open('data/ds-'+i+'.npy', 'rb') as f:
        Y[i]['input'] = numpy.load(f)
        X[i]['input'] = numpy.load(f)
        Z[i]['input'] = numpy.load(f, allow_pickle=True)

# load in the results for SuspectPZA
suspectpza={}
for i in ['validation-samples', 'validation-mutations','mic']:
    suspectpza[i]={}
    with open('data/suspectpza-'+i+'.npy', 'rb') as f:
        suspectpza[i]['input'] = numpy.load(f)
        suspectpza[i]['predicted'] = numpy.load(f)
        suspectpza[i]['muts'] = numpy.load(f, allow_pickle=True)
line = []

/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/fowler/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.3.0 wh

For comparison later, let's make a subset of `validation-samples` that exclude those mutations which have an inconsistent phenotype, either because too few are in the dataset or they do not consistently test as R or S, suggesting they may be near the breakpoint/ECOFF. 

We need to do this for both the `validation-samples` features and the results from the published SuspectPZA model.

In [3]:
df = pandas.read_csv('data/ds-validation-mutations-full.csv')

mask = ~numpy.isin(Z['validation-samples']['input'], df[df.CONSISTENT_PHENOTYPE=='U'].MUTATION.unique())

X['validation-samplesnoU']={}
Y['validation-samplesnoU']={}
Z['validation-samplesnoU']={}

X['validation-samplesnoU']['input']=X['validation-samples']['input'][mask]
Y['validation-samplesnoU']['input']=Y['validation-samples']['input'][mask]
Z['validation-samplesnoU']['input']=Z['validation-samples']['input'][mask]

mask = ~numpy.isin(suspectpza['validation-samples']['muts'],(df[df.CONSISTENT_PHENOTYPE=='U'].MUTATION.unique()))
suspectpza['validation-samplesnoU']={}
suspectpza['validation-samplesnoU']['input']=suspectpza['validation-samples']['input'][mask]
suspectpza['validation-samplesnoU']['predicted']=suspectpza['validation-samples']['predicted'][mask]
suspectpza['validation-samplesnoU']['muts']=suspectpza['validation-samples']['muts'][mask]

In [4]:
df[df.CONSISTENT_PHENOTYPE!='U'].MUTATION.unique()

array(['A102R', 'A102T', 'A134V', 'A143G', 'A143T', 'A143V', 'A146E',
       'A146P', 'A146T', 'A146V', 'A171V', 'A30V', 'A3E', 'A46E', 'A46T',
       'A79T', 'A79V', 'C138R', 'C14R', 'C72R', 'C72Y', 'D129Y', 'D12A',
       'D12E', 'D12N', 'D136Y', 'D49A', 'D49G', 'D49N', 'D63A', 'D8A',
       'D8E', 'D8G', 'D8N', 'E15G', 'F106S', 'F13I', 'F13V', 'F58V',
       'F81C', 'F81S', 'F94C', 'F94S', 'G105D', 'G105V', 'G108R', 'G124D',
       'G132A', 'G132C', 'G132D', 'G132S', 'G162D', 'G17D', 'G17S',
       'G24D', 'G97C', 'G97D', 'G97R', 'G97S', 'G97V', 'H137P', 'H137Q',
       'H43P', 'H43Y', 'H51D', 'H51P', 'H51Q', 'H51R', 'H51Y', 'H57D',
       'H57L', 'H57P', 'H57Q', 'H57R', 'H57Y', 'H71P', 'H71Q', 'H71Y',
       'H82D', 'H82R', 'I31S', 'I5S', 'I5T', 'I6L', 'I6S', 'I6T', 'I6V',
       'I90T', 'K48N', 'K96E', 'K96N', 'K96R', 'K96T', 'L116R', 'L120P',
       'L120Q', 'L120R', 'L151S', 'L156P', 'L159R', 'L172P', 'L172R',
       'L182F', 'L19P', 'L27P', 'L35R', 'L4S', 'L85P', 'L85R', 'M175I

The below function takes a supplied model, applies it to the features of the different datasets and measures a range of metrics we can use to evaluate their performance.

Note that this uses the separate `construct_line` function which can be found in `misc.py`.

In [5]:
def validate_model(line, best_model, model_name, X, Y):

    for dataset in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']: 
        
        Y[dataset]['predicted'] = best_model.predict(X[dataset]['input'])
        Y[dataset]['scores'] = best_model.predict_proba(X[dataset]['input'])[:,1]

        row = construct_line(model_name , dataset, None, Y[dataset], None)
        line.append(row)

    return(line)

In [6]:
for model in ['NN', 'XB', 'LR']:
    line = validate_model(line, best_model[model], model, X, Y)

In [7]:
for i in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
    line.append(construct_line('SP', i, None, suspectpza[i], None))

In [8]:
def bootstrap_model(line, best_model, model_name, X, Y):
    x={}
    y={}
    for dataset in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
        print(model_name, dataset)
        for i in range(number_of_bootstraps):
            x['input'],y['input'] = resample(X[dataset]['input'], Y[dataset]['input'])
            y['predicted'] = best_model.predict(x['input'])
            y['scores'] = best_model.predict_proba(x['input'])[:,1]
            row = construct_line(model_name, dataset+'_'+str(i), None, y, None)
            line.append(row)
    return(line)

In [9]:
for model in ['LR', 'NN', 'XB']:
    line = bootstrap_model(line, best_model[model], model, X, Y)

LR validation-samples
LR validation-samplesnoU
LR validation-mutations
LR mic
NN validation-samples
NN validation-samplesnoU
NN validation-mutations
NN mic
XB validation-samples
XB validation-samplesnoU
XB validation-mutations
XB mic


In [10]:
test_results = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])
test_results[:3]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
0,NN,validation-samples,90.517241,None,51.649236,None,73.731156,None,642,601,264,2520,null
1,NN,validation-samplesnoU,91.479239,None,66.666667,None,80.821306,None,538,269,197,2115,null
2,NN,validation-mutations,94.193548,None,52.272727,None,77.565982,None,23,21,9,146,null


In [11]:
line = []
for i in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
    for model in ['LR', 'NN', 'XB']:
        sens_mean = test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].sensitivity_mean.mean()
        sens_std = 1.96*test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].sensitivity_mean.std()/(number_of_bootstraps**0.5)
        spec_mean = test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].specificity_mean.mean()
        spec_std = 1.96*test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].specificity_mean.std()/(number_of_bootstraps**0.5)
        roc_mean = test_results[(test_results.model==model) & (test_results.dataset!=i) & (test_results.dataset.str.contains(i+"_"))].roc_auc_mean.mean()
        row = [model, 'bootstrapped-'+i, sens_mean, sens_std, spec_mean, spec_std, roc_mean, None, None, None, None, None, None ]
        line.append(row)

extra_rows = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])
test_results  = pandas.concat([test_results, extra_rows])
test_results[:3]

/var/folders/dh/1k1mg9v14wd4bhfwdyhp5ygh0000gn/T/ipykernel_26344/853011173.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_results  = pandas.concat([test_results, extra_rows])


,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
0,NN,validation-samples,90.517241,NaN,51.649236,NaN,73.731156,None,642,601,264,2520,null
1,NN,validation-samplesnoU,91.479239,NaN,66.666667,NaN,80.821306,None,538,269,197,2115,null
2,NN,validation-mutations,94.193548,NaN,52.272727,NaN,77.565982,None,23,21,9,146,null


In [12]:
extra_rows


,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
0,LR,bootstrapped-validation-samples,98.099271,0.158352,39.231420,0.752355,79.118830,None,None,None,None,None,None
1,NN,bootstrapped-validation-samples,90.354093,0.307364,51.841949,0.767843,74.000388,None,None,None,None,None,None
2,XB,bootstrapped-validation-samples,97.227939,0.140828,46.114237,0.943217,80.920663,None,None,None,None,None,None
3,LR,bootstrapped-validation-samplesnoU,98.435990,0.185805,55.402843,1.385556,84.687786,None,None,None,None,None,None
4,NN,bootstrapped-validation-samplesnoU,91.284282,0.408183,67.701865,1.333477,81.360994,None,None,None,None,None,None
5,XB,bootstrapped-validation-samplesnoU,97.670088,0.193840,62.536386,1.200973,86.817855,None,None,None,None,None,None
6,LR,bootstrapped-validation-mutations,97.911428,0.856200,44.803421,3.266906,84.772404,None,None,None,None,None,None
7,NN,bootstrapped-validation-mutations,93.384002,1.411193,51.474827,5.883973,77.135948,None,None,None,None,None,None
8,XB,bootstrapped-validation-mutations,97.801821,0.505757,58.493078,3.145322,88.516954,None,None,None,None,None,None
9,LR,bootstrapped-mic,100.000000,0.000000,0.000000,0.000000,65.490509,None,None,None,None,None,None


In [13]:
test_results[(test_results.model=='XB') & (test_results.dataset!='validation-samplesnoU') & (test_results.dataset.str.contains('validation-samplesnoU'+"_"))]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
106,XB,validation-samplesnoU_0,98.035867,NaN,64.221364,NaN,87.605194,None,499,278,46,2296,null
107,XB,validation-samplesnoU_1,97.140418,NaN,63.530928,NaN,87.918113,None,493,283,67,2276,null
108,XB,validation-samplesnoU_2,97.519582,NaN,64.068210,NaN,86.698543,None,526,295,57,2241,null
109,XB,validation-samplesnoU_3,98.081116,NaN,63.680387,NaN,87.909566,None,526,300,44,2249,null
110,XB,validation-samplesnoU_4,97.663349,NaN,60.024752,NaN,85.051717,None,485,323,54,2257,null
111,XB,validation-samplesnoU_5,97.792208,NaN,60.692213,NaN,87.363454,None,491,318,51,2259,null
112,XB,validation-samplesnoU_6,97.793163,NaN,60.272277,NaN,85.322778,None,487,321,51,2260,null
113,XB,validation-samplesnoU_7,97.188581,NaN,64.931846,NaN,87.882895,None,524,283,65,2247,null
114,XB,validation-samplesnoU_8,97.772065,NaN,63.566879,NaN,87.059775,None,499,286,52,2282,null
115,XB,validation-samplesnoU_9,97.714532,NaN,60.375000,NaN,85.366510,None,483,317,53,2266,null


In [14]:
for dataset in ['validation-samples', 'validation-samplesnoU', 'validation-mutations', 'mic']:
    for metric in ['sensitivity_mean', 'specificity_mean']:
        for i in ['XB', 'NN', 'LR']:
            for j in ['XB', 'NN', 'LR']:
                if i!=j:
                    a = numpy.array(test_results[(test_results.model==i)& (test_results.dataset.str.contains(dataset+'_'))][metric])
                    b = numpy.array(test_results[(test_results.model==j)& (test_results.dataset.str.contains(dataset+'_'))][metric])
                    assert len(a) == number_of_bootstraps
                    tstat, pvalue = ztest(a,b)
                    if pvalue < 0.05:
                        print(dataset,metric, i,j,"Signficant",pvalue)
        print()

validation-samples sensitivity_mean XB NN Signficant 0.0
validation-samples sensitivity_mean XB LR Signficant 7.695943826508443e-16
validation-samples sensitivity_mean NN XB Signficant 0.0
validation-samples sensitivity_mean NN LR Signficant 0.0
validation-samples sensitivity_mean LR XB Signficant 7.695943826508443e-16
validation-samples sensitivity_mean LR NN Signficant 0.0

validation-samples specificity_mean XB NN Signficant 2.6977786470061497e-20
validation-samples specificity_mean XB LR Signficant 5.0426413981379467e-29
validation-samples specificity_mean NN XB Signficant 2.6977786470061497e-20
validation-samples specificity_mean NN LR Signficant 5.5716556626597075e-117
validation-samples specificity_mean LR XB Signficant 5.0426413981379467e-29
validation-samples specificity_mean LR NN Signficant 5.5716556626597075e-117

validation-samplesnoU sensitivity_mean XB NN Signficant 7.26011503844793e-169
validation-samplesnoU sensitivity_mean XB LR Signficant 2.2609475995053583e-08
valid

/Users/fowler/Library/Python/3.11/lib/python/site-packages/statsmodels/stats/weightstats.py:748: RuntimeWarning: invalid value encountered in scalar divide
  zstat = (value1 - value2 - diff) / std_diff


In [15]:
# test_results = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])

# calculate the diagnostic odds ration
# test_results['diagnostic_odds_ratio_mean'] = (test_results['TN']*test_results['TP'])/(test_results['FN']*test_results['FP'])
test_results['diagnostic_odds_ratio_mean'] = 0
test_results['diagnostic_odds_ratio_std'] = 0

# save to disc as a CSV
test_results.to_csv('results-validation.csv', index=False)

test_results[:20]

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters,diagnostic_odds_ratio_mean,diagnostic_odds_ratio_std
0,NN,validation-samples,90.517241,NaN,51.649236,NaN,73.731156,None,642,601,264,2520,null,0,0
1,NN,validation-samplesnoU,91.479239,NaN,66.666667,NaN,80.821306,None,538,269,197,2115,null,0,0
2,NN,validation-mutations,94.193548,NaN,52.272727,NaN,77.565982,None,23,21,9,146,null,0,0
3,NN,mic,94.000000,NaN,42.857143,NaN,63.142857,None,3,4,3,47,null,0,0
4,XB,validation-samples,97.198276,NaN,46.017699,NaN,80.722592,None,572,671,78,2706,null,0,0
5,XB,validation-samplesnoU,97.621107,NaN,63.073110,NaN,86.880823,None,509,298,55,2257,null,0,0
6,XB,validation-mutations,97.419355,NaN,59.090909,NaN,88.284457,None,26,18,4,151,null,0,0
7,XB,mic,100.000000,NaN,14.285714,NaN,67.714286,None,1,6,0,50,null,0,0
8,LR,validation-samples,98.060345,NaN,40.305712,NaN,79.642377,None,501,742,54,2730,null,0,0
9,LR,validation-samplesnoU,98.442907,NaN,56.133829,NaN,84.841520,None,453,354,36,2276,null,0,0


In [16]:
test_results.model.value_counts()

model
NN    48
XB    48
LR    48
SP     4
Name: count, dtype: int64

Finally let's see how well the published `SuspectPZA` models does on these validtion datasets

In [17]:
test_results[test_results.model=='SP']

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters,diagnostic_odds_ratio_mean,diagnostic_odds_ratio_std
12,SP,validation-samples,93.678161,NaN,44.328238,NaN,NaN,None,551,692,176,2608,null,0,0
13,SP,validation-samplesnoU,96.107266,NaN,57.372986,NaN,NaN,None,463,344,90,2222,null,0,0
14,SP,validation-mutations,92.258065,NaN,47.727273,NaN,NaN,None,21,23,12,143,null,0,0
15,SP,mic,92.000000,NaN,14.285714,NaN,NaN,None,1,6,4,46,null,0,0
